In [2]:
import undetected_chromedriver as uc
from pyvirtualdisplay import Display

Need to understand how pyvirtualdisplay works... 

In [3]:
display = Display(visible=0, size=(800, 600))
display.start()

options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
wd = uc.Chrome(options=options)

In [15]:
display.is_alive()

True

In [4]:
def extract_text_by_class(c):
    global wd
    try:
        content = wd.find_element(By.CLASS_NAME, c)
        return content.text
    except:
        return ""

In [5]:
def extract_links_by_xpath(xpath):
    #Note the global variable is being called into the function
    global wd
    links = set()
    try:
        a_elems = wd.find_elements(By.XPATH, xpath)
        for elem in a_elems:
            link = elem.get_attribute("href")
            if link == "javascript:void(0)":
                continue
            # Remove links to images and various files
            if (
                link.endswith(".png")
                or link.endswith(".json")
                or link.endswith(".txt")
                or link.endswith(".svg")
                or link.endswith(".ipynb")
                or link.endswith(".jpg")
                or link.endswith(".pdf")
                or link.endswith(".mp4")
                or "mailto" in link
                or len(link) > 300
            ):
                continue
            # Remove anchors
            link = link.split("#")[0]
            # Remove parameters
            link = link.split("?")[0]
            # Remove trailing forward slash
            link = link.rstrip("/")
            links.add(link)
        return list(links)
    except:
        return []

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("yanekyuk/bert-uncased-keyword-extractor")

In [9]:
model = AutoModelForTokenClassification.from_pretrained(pretrained_model_name_or_path="yanekyuk/bert-uncased-keyword-extractor",
                                                       resume_download=True)

In [10]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

def extract_keywords(text):
    """
    Extract keywords and construct them back from tokens
    """
    result = list()
    keyword = ""
    for token in nlp(text):
        if token["entity"] == "I-KEY":
            keyword += (
                token["word"][2:]
                if token["word"].startswith("##")
                else f" {token['word']}"
            )
        else:
            if keyword:
                result.append(keyword)
            keyword = token["word"]
    # Add the last keyword
    result.append(keyword)
    return list(set(result))

In [11]:
extract_keywords(
    """
Broadcom agreed to acquire cloud computing company VMware in a $61 billion (€57bn) cash-and stock deal.
"""
)

['broadcom', 'vmware', 'cloud computing']

In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def generate_embeddings(text):
    embeddings = model.encode(text)
    return [float(x) for x in embeddings.tolist()]

In [13]:
from selenium.webdriver.common.by import By

entry_url = "https://neo4j.com/docs"
data = dict()
visit_list = [entry_url]
already_visited = []

In [34]:
entry_url = "http://tutorkamal.pythonanywhere.com/courses/"

In [22]:
current_url = visit_list[0]
wd.get(visit_list[0])

In [16]:
# Extract text from the content div
text = extract_text_by_class("content")
# If nothing is found, try article div
if not text:
    text = extract_text_by_class("article")
# If nothing is found, try page div
if not text:
    text = extract_text_by_class("page")
if not text:
    text = extract_text_by_class("single-user-story")

In [17]:
text

'Neo4j documentation\nNeo4j DBMS\nLearn about graph database concepts, introduce yourself to Cypher, and find useful resources.\nGet started\nNeo4j Aura\nLearn how to create accounts, manage databases, and access Aura via cloud providers.\nAuraDB AuraDS\nCypher\nLearn about Cypher; the graph query language for Neo4j and AuraDB.\nCypher docs Cypher Cheat Sheet\nAPOC\nDiscover the APOC library, now supported in Neo4j version 5.\nAPOC User Guide\nUpgrade or migrate\nLearn how to keep your Neo4j deployment up-to-date.\nUpgrade / Migrate\nCreate applications\nDiscover the drivers, libraries, connectors, and APIs to develop applications with Neo4j and AuraDB.\nCreate applications with Neo4j\nNeo4j Graph Data Science\nFor data scientists and analysts interested in scalable analytics powered by graph algorithms and machine learning models.\nGDS library GDS client AuraDS\nNeo4j Bloom and other tooling\nLearn to explore your graph with Neo4j Bloom and other tools, such as Neo4j Desktop, Neo4j Br

In [35]:
#The extraction fn is hard coded to the classes text data, 
#which will not work for all the site
wd.get(entry_url)

In [42]:
para="//p"

para_content = wd.find_elements(By.XPATH, para)

In [48]:
text = ''
for t in para_content:
    text = text + t.text

In [49]:
text

"Technology solves challenges and improves the quality of life across the world. Understanding how each part of the Technology does it is what we teach in our courses500 INRSeries of lectures that provide a practical look at how Django framework can be mastered, and used for real websites in minimal time and effort750 INRIn this curated collection, we delve into the fascinating world of generative AI, focusing on the crucial aspects of hardware and software. Discover the cutting-edge technologies, tools, and techniques that power the creation of artificial …958 INRHello and welcome to this Course about Hugging Face AI models and their use cases. Hugging Face is a leading company in the development and deployment of natural language processing (NLP) models, which are designed …750 INRCourse videos on virtual environments available in python ecosystem their advantages, challenges and usage750 INRPython is one of the most popular programming languages today, and its popularity is mainly d

In [18]:
embeddings = generate_embeddings(text)
keywords = extract_keywords(text)

In [19]:
keywords

['database', 'aura', 'graph', 'query language']

In [20]:
links = extract_links_by_xpath("//div[@class='content']//a[@href]")
    # If nothing is found, try article div
if not links:
    links = extract_links_by_xpath("//article[@class='article']//a[@href]")
if not links:
    links = extract_links_by_xpath("//article//a[@href]")

In [21]:
links

['https://neo4j.com/docs/getting-started',
 'https://neo4j.com/docs/graph-data-science-client',
 'https://neo4j.com/docs/cypher-cheat-sheet',
 'https://neo4j.com/docs/graph-data-science',
 'https://neo4j.com/docs/security-docs',
 'https://neo4j.com/docs',
 'https://neo4j.com/docs/upgrade-migration-guide',
 'https://neo4j.com/docs/create-applications',
 'https://neo4j.com/docs/aura/auradb',
 'https://neo4j.com/docs/apoc/current',
 'https://neo4j.com/docs/developer-tools',
 'https://neo4j.com/docs/license',
 'https://neo4j.com/docs/cypher-manual',
 'https://neo4j.com/docs/aura/aurads']

In [23]:
data[current_url] = {
    "links": [l for l in links if l != current_url],
    "text": text,
    "embeddings": embeddings,
    "keywords": keywords,
    "redirects": [],
}

In [24]:
data

{'https://neo4j.com/docs': {'links': ['https://neo4j.com/docs/getting-started',
   'https://neo4j.com/docs/graph-data-science-client',
   'https://neo4j.com/docs/cypher-cheat-sheet',
   'https://neo4j.com/docs/graph-data-science',
   'https://neo4j.com/docs/security-docs',
   'https://neo4j.com/docs/upgrade-migration-guide',
   'https://neo4j.com/docs/create-applications',
   'https://neo4j.com/docs/aura/auradb',
   'https://neo4j.com/docs/apoc/current',
   'https://neo4j.com/docs/developer-tools',
   'https://neo4j.com/docs/license',
   'https://neo4j.com/docs/cypher-manual',
   'https://neo4j.com/docs/aura/aurads'],
  'text': 'Neo4j documentation\nNeo4j DBMS\nLearn about graph database concepts, introduce yourself to Cypher, and find useful resources.\nGet started\nNeo4j Aura\nLearn how to create accounts, manage databases, and access Aura via cloud providers.\nAuraDB AuraDS\nCypher\nLearn about Cypher; the graph query language for Neo4j and AuraDB.\nCypher docs Cypher Cheat Sheet\nA

In [ ]:
actual_url = wd.current_url.rstrip("/").split("#")[0].split("?")[0]
#This code executes if the new url is in fact completely new and 
#not just redirection
if actual_url != current_url:
    # Store the redirect information
    data[current_url] = {
        "links": [],
        "text": None,
        "embeddings": [],
        "keywords": [],
        "redirects": [actual_url],
    }
    already_visited.append(current_url)
    # Content is being extracted from the actual url
    current_url = actual_url

In [ ]:
while visit_list:
    # Visit the URL
    current_url = visit_list.pop()
    if current_url in already_visited:
        continue
    #print(current_url)
    try:
        wd.get(current_url)
    except:
        print(f"Couldn't open {current_url}")
        already_visited.append(current_url)
        display = Display(visible=0, size=(800, 600))
        display.start()

        options = uc.ChromeOptions()
        options.add_argument("--no-sandbox")
        wd = uc.Chrome(options=options)
        continue
    # If redirect
    try:
        actual_url = wd.current_url.rstrip("/").split("#")[0].split("?")[0]
        if actual_url != current_url:
            # Store the redirect information
            data[current_url] = {
                "links": [],
                "text": None,
                "embeddings": [],
                "keywords": [],
                "redirects": [actual_url],
            }
            already_visited.append(current_url)
            # Content is being extracted from the actual url
            current_url = actual_url
    except:
        pass
    # Extract text from the content div
    text = extract_text_by_class("content")
    # If nothing is found, try article div
    if not text:
        text = extract_text_by_class("article")
    # If nothing is found, try page div
    if not text:
        text = extract_text_by_class("page")
    if not text:
        text = extract_text_by_class("single-user-story")
    # Check if 404
    try:
        if "Sorry, page not found" in wd.find_element(By.TAG_NAME, "body").text:
            text = "404"
    except:
        pass

    # Generate paragraph embedding & extract keywords
    if text:
        embeddings = generate_embeddings(text)
        keywords = extract_keywords(text)
    else:
        embeddings = []
        keywords = []

    # Extract links from the content div
    links = extract_links_by_xpath("//div[@class='content']//a[@href]")
    # If nothing is found, try article div
    if not links:
        links = extract_links_by_xpath("//article[@class='article']//a[@href]")
    if not links:
        links = extract_links_by_xpath("//article//a[@href]")

    # if not links and not text:
    #    print(f"Couldn't retrieve the data from {current_url}")
    # Store page information
    data[current_url] = {
        "links": [l for l in links if l != current_url],
        "text": text,
        "embeddings": embeddings,
        "keywords": keywords,
        "redirects": [],
    }
    # Crawling information
    already_visited.append(current_url)
    # Don't leave neo4j.com while crawling
    # and avoid community and sandbox
    visit_list.extend(
        [
            l
            for l in list(links)
            if ("neo4j.com" in l)
            and (not l in already_visited)
            and (not "community.neo4j.com" in l)
            and (not "sandbox.neo4j.com" in l)
        ]
    )

In [25]:
import json

j = json.dumps(data)
# open file for writing, "w"
f = open("neo4j_docs.json", "w")

# write json object to file
f.write(j)

# close file
f.close()

In [50]:
# Import to Neo4j
from graphdatascience import GraphDataScience

host = "bolt://0.0.0.0:7687"
user = "neo4j"
password = "1234ehtd"

In [51]:
gds = GraphDataScience(host, auth=(user, password))

In [52]:
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (p:Page) REQUIRE p.url IS UNIQUE;
"""
)

""


In [53]:
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (k:Keyword) REQUIRE k.name IS UNIQUE;
"""
)

""


In [54]:
import_query = """

UNWIND $data AS row
MERGE (p:Page {url:row.url})
SET p.embedding = row.embedding,
    p.has_text = row.has_text,
    p.is_404 = row.is_404
FOREACH (l in row.links    | MERGE (p1:Page {url:l}) MERGE (p)-[:LINKS_TO]->(p1))
FOREACH (k in row.keywords | MERGE (k1:Keyword {name:k}) MERGE (p)-[:HAS_KEYWORD]->(k1))
FOREACH (r in row.redirects| MERGE (r1:Page {url:r}) MERGE (p)-[:REDIRECTS]->(r1))

"""

In [55]:
x = 1
params = []
for key in data:
    params.append(
        {
            "url": key,
            "embedding": data[key]["embeddings"],
            "keywords": data[key]["keywords"],
            "links": data[key]["links"],
            "has_text": True if data[key]["text"] else False,
            "is_404": True if data[key]["text"] == "404" else False,
            "redirects": data[key]["redirects"],
        }
    )
    # Batch per 500
    if len(params) == 500:
        gds.run_cypher(import_query, {"data": params})
        params = []
        # Logging
        print(f"Importing {x} batch")
        x += 1

In [56]:

# Import the remainder
gds.run_cypher(import_query, {"data": params})

""


In [57]:
gds.run_cypher("""
CALL apoc.meta.stats()
YIELD labels, relTypesCount
""")

,labels,relTypesCount
0,"{'Keyword': 4, 'Page': 14}","{'LINKS_TO': 13, 'HAS_KEYWORD': 4}"


In [58]:
gds.run_cypher("""
MATCH (p:Page)
RETURN p.has_text AS has_text,
       count(*) AS count
""")

,has_text,count
0,True,1
1,None,13


In [59]:
gds.run_cypher("""
MATCH (p:Page)
WHERE p.has_text IS NULL
RETURN p.url AS page,
       count{(p)<-[:LINKS_TO|REDIRECTS]-()} AS links
ORDER BY links DESC
LIMIT 5
""")

,page,links
0,https://neo4j.com/docs/graph-data-science-client,1
1,https://neo4j.com/docs/cypher-cheat-sheet,1
2,https://neo4j.com/docs/graph-data-science,1
3,https://neo4j.com/docs/security-docs,1
4,https://neo4j.com/docs/getting-started,1


In [60]:
gds.run_cypher("""
MATCH (:Page)-[:LINKS_TO|REDIRECTS]->(:Page{is_404:true})
RETURN count(*) AS brokenLinkCount
""")

,brokenLinkCount
0,0
